In [2]:
import os
import numpy as np
import mne
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import copy
from mne.channels import find_ch_adjacency
%matplotlib qt5

# Funciones  análisis de clústeres

In [3]:
def matrices_cluster(grupo):
    # Diccionario con evocados de todos los sujetos, para ensayos congruentes e incongruentes
    congruentes = copy.deepcopy(evocados_co)
    incongruentes = copy.deepcopy(evocados_in)
    tiempos = congruentes['22100'].times
    adjacency, canales = find_ch_adjacency(congruentes['22100'].info, "eeg")

    # Matrices de evocados (tiempos, canales) de sujetos condición 1
    grupo_co = {}
    grupo_in = {}
    for sujeto in grupo:
        grupo_co[sujeto] = congruentes[sujeto]
        grupo_in[sujeto] = incongruentes[sujeto]

    ga_grupo_co = mne.grand_average(list(grupo_co.values()), interpolate_bads=True, drop_bads=True)
    ga_grupo_in = mne.grand_average(list(grupo_in.values()), interpolate_bads=True, drop_bads=True)

    co_grupo = [grupo_co[i].data for i in grupo_co]
    co_grupo = np.reshape(co_grupo,(-1,len(canales), len(tiempos)))
    co_grupo = np.transpose(co_grupo, axes=(0,2,1))
    print(co_grupo.shape)

    in_grupo = [grupo_in[i].data for i in grupo_in]
    in_grupo = np.reshape(in_grupo,(-1,len(canales), len(tiempos)))
    in_grupo = np.transpose(in_grupo, axes=(0,2,1))
    print(in_grupo.shape)

    return (ga_grupo_co, ga_grupo_in, co_grupo, in_grupo, tiempos, canales, adjacency)

In [4]:
# Evocados de clústeres
def ROI_evoked(evocado, canales):
    lista_canales = evocado.ch_names
    lista_indices = []
    for canal in canales:
        indice = lista_canales.index(canal)
        lista_indices.append(indice)
    ROI_evocado = mne.channels.combine_channels(evocado, groups={'ROI':lista_indices})
    return ROI_evocado

# Cálculo de media de clúster
def medidas(evoked, tmin, tmax):
    evoked_ROI = evoked.copy()
    evoked_ROI.crop(tmin = tmin, tmax = tmax)
    data = evoked_ROI.to_data_frame(time_format = None)
    data.set_index('time',inplace=True)
    mean_ROI = data.mean()
    return mean_ROI

In [5]:
# Diccionario con evocados de todos los sujetos, para ensayos congruentes e incongruentes
rutas_archivos = []
ruta_dir = os.getcwd() + '\\Sin filtrar\\'
dir = os.listdir(ruta_dir)
for q in range(len(dir)):
    ruta_arc =ruta_dir + dir[q]
    rutas_archivos.append(ruta_arc)

evocados_co = {}
evocados_in = {}

for l in range(len(rutas_archivos)):
    ruta=rutas_archivos[l]
    epochs = mne.read_epochs(ruta)
    epochs.filter(l_freq=None, h_freq=30)
    evoked_co = epochs['12','13','14','15','16','17','18','19'].average()
    evocados_co[ruta[-14:-9]] = evoked_co
    evoked_in = epochs['24','25','26','27','28','29','30','31'].average()
    evocados_in[ruta[-14:-9]] = evoked_in

Reading c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\Sin filtrar\21100n_epo.fif ...
    Found the data of interest:
        t =    -199.22 ...     796.88 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
145 matching events found
No baseline correction applied
0 projection items activated
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 113 samples (0.441 sec)

Reading c:\Users\jhquiza\OneDrive - Universidad de Medellin\JupyterNotebooks\IAT\Sin filtrar\21101n_epo.fif ...
    Found the data of interest:
        t =    -199.22 ...     796.88 ms
        0 CTF compensation matrices available

In [6]:
evocados_co.keys()

dict_keys(['21100', '21101', '21102', '21103', '21104', '21105', '21106', '21107', '21108', '21109', '21110', '21111', '21112', '21113', '21114', '21115', '21116', '21118', '21121', '21122', '21123', '21125', '21127', '21129', '21131', '21132', '21134', '21135', '21137', '21139', '21140', '21143', '21144', '21147', '21152', '22100', '22101', '22102', '22103', '22104', '22106', '22107', '22108', '22110', '22111', '22112', '22113', '22114', '22115', '22116', '22117', '23001', '23002', '23003', '23004', '23005', '23006', '23007', '23008', '23009', '23010', '23011', '23012', '23013', '23015', '23016', '23017', '23018', '23019', '23020', '23021', '23022', '23023', '23025', '24002', '24003', '24004', '24005', '24013', '24014', '24015', '24017', '24025', '24026', '24027', '24035', '24039', '24043', '24045', '24052', '24053', '24057', '24059', '24065', '24069', '24073', '24101'])

# Gráficas excombatientes, víctimas

In [7]:
df = pd.read_csv('Datos_IAT_comportamentales_personales.csv', index_col='subject')
df.head()

,type,exposure_level,group,victim_self,victim_self_id,age,scholarship,gender,laterality,dscore_0,...,DAP,AAP,TPA,TPV,EASPA,ANA,BN,ANC,AGA.1,TAPV
subject,,,,,,,,,,,,,,,,,,,,,
21100,excombatant,high,exguerrilla,yes,yes,19,11,F,D,-0.197633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21101,excombatant,high,exparamilitar,yes,yes,46,11,M,D,-0.116918,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
21102,excombatant,low,exparamilitar,yes,yes,31,11,M,D,0.473119,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21103,excombatant,high,exparamilitar,yes,yes,42,18,M,D,0.214867,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21104,excombatant,high,exparamilitar,yes,yes,38,11,M,D,0.423545,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
excombatant = list(df[(df['type']=='excombatant')&(df['group']!='ex-both-sides')].index)
victim = list(df[df['type']=='victim'].index)
exparamilitar = list(df[df['group']=='exparamilitar'].index)
exguerrilla = list(df[df['group']=='exguerrilla'].index)
print('count excombatants: ',len(excombatant),'\ncount victims: ',len(victim),'\ncount exparamilitar: ',len(exparamilitar),'\ncount exguerrilla: ',len(exguerrilla))

count excombatants:  53 
count victims:  23 
count exparamilitar:  31 
count exguerrilla:  22


In [12]:
for i in range(len(excombatant)):
    excombatant[i] = str(excombatant[i])
for i in range(len(victim)):
    victim[i] = str(victim[i])
for i in range(len(exparamilitar)):
    exparamilitar[i] = str(exparamilitar[i])
for i in range(len(exguerrilla)):
    exguerrilla[i] = str(exguerrilla[i])

__, __, excombatant_co, excombatant_in, __, __, __ = matrices_cluster(excombatant)
__, __, victim_co, victim_in, __, __, __ = matrices_cluster(victim)
__, __, exparamilitar_co, exparamilitar_in, __, __, __ = matrices_cluster(exparamilitar)
__, __, exguerrilla_co, exguerrilla_in, tiempos, canales, adjacency = matrices_cluster(exguerrilla)

# Matrices de diferencias entre ensayos
excombatant_dif = excombatant_co - excombatant_in
victim_dif = victim_co - victim_in
exparamilitar_dif = exparamilitar_co - exparamilitar_in
exguerrilla_dif = exguerrilla_co - exguerrilla_in

Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(53, 256, 64)
(53, 256, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(23, 256, 64)
(23, 256, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(31, 256, 64)
(31, 256, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(22, 256, 64)
(22, 256, 64)
(53, 256, 64)


In [13]:
def lista_evocados_grupos(canales, tmin=0.0, tmax=0.796875):
    excombatant_co = {}
    excombatant_in = {}
    excombatant_dif = {}
    for sujeto in excombatant:
        excombatant_co[sujeto] = ROI_evoked(evocados_co[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        excombatant_in[sujeto] = ROI_evoked(evocados_in[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        data_dif = excombatant_co[sujeto].data - excombatant_in[sujeto].data
        excombatant_dif[sujeto] = mne.EvokedArray(data=data_dif, info=excombatant_in[sujeto].info, tmin=excombatant_in[sujeto].tmin)
    victim_co = {}
    victim_in = {}
    victim_dif = {}
    for sujeto in victim:
        victim_co[sujeto] = ROI_evoked(evocados_co[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        victim_in[sujeto] = ROI_evoked(evocados_in[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        data_dif = victim_co[sujeto].data - victim_in[sujeto].data
        victim_dif[sujeto] = mne.EvokedArray(data=data_dif, info=victim_in[sujeto].info, tmin=victim_in[sujeto].tmin)
    exparamilitar_co = {}
    exparamilitar_in = {}
    exparamilitar_dif = {}
    for sujeto in exparamilitar:
        exparamilitar_co[sujeto] = ROI_evoked(evocados_co[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        exparamilitar_in[sujeto] = ROI_evoked(evocados_in[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        data_dif = exparamilitar_co[sujeto].data - exparamilitar_in[sujeto].data
        exparamilitar_dif[sujeto] = mne.EvokedArray(data=data_dif, info=exparamilitar_in[sujeto].info, tmin=exparamilitar_in[sujeto].tmin)
    exguerrilla_co = {}
    exguerrilla_in = {}
    exguerrilla_dif = {}
    for sujeto in exguerrilla:
        exguerrilla_co[sujeto] = ROI_evoked(evocados_co[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        exguerrilla_in[sujeto] = ROI_evoked(evocados_in[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        data_dif = exguerrilla_co[sujeto].data - exguerrilla_in[sujeto].data
        exguerrilla_dif[sujeto] = mne.EvokedArray(data=data_dif, info=exguerrilla_in[sujeto].info, tmin=exguerrilla_in[sujeto].tmin)
    return excombatant_co, excombatant_in, excombatant_dif, victim_co, victim_in, victim_dif, exparamilitar_co, exparamilitar_in, exparamilitar_dif, exguerrilla_co, exguerrilla_in, exguerrilla_dif

In [29]:
def erps_grupos(canales, tmin=0.0, tmax=0.796875):
    # Diccionarios de evocados de clústeres por grupos
    excombatant_co, excombatant_in, excombatant_dif, victim_co, victim_in, victim_dif, exparamilitar_co, exparamilitar_in, exparamilitar_dif, exguerrilla_co, exguerrilla_in, exguerrilla_dif = lista_evocados_grupos(canales=canales, tmin=tmin, tmax=tmax)
        
    # Diccionarios de toda la muestra
    ROI_co_type = {**excombatant_co,**victim_co}
    ROI_in_type = {**excombatant_in,**victim_in}
    ROI_dif_type = {**excombatant_dif,**victim_dif}
    ROI_co = {**exparamilitar_co,**exguerrilla_co,**victim_co}
    ROI_in = {**exparamilitar_in,**exguerrilla_in,**victim_in}
    ROI_dif = {**exparamilitar_dif,**exguerrilla_dif,**victim_dif}
    
    # Listas de evocados de clústeres por grupos
    excombatant_co_list = list(excombatant_co.values())
    excombatant_in_list = list(excombatant_in.values())
    excombatant_dif_list = list(excombatant_dif.values())
    victim_co_list = list(victim_co.values())
    victim_in_list = list(victim_in.values())
    victim_dif_list = list(victim_dif.values())
    exparamilitar_co_list = list(exparamilitar_co.values())
    exparamilitar_in_list = list(exparamilitar_in.values())
    exparamilitar_dif_list = list(exparamilitar_dif.values())
    exguerrilla_co_list = list(exguerrilla_co.values())
    exguerrilla_in_list = list(exguerrilla_in.values())
    exguerrilla_dif_list = list(exguerrilla_dif.values())
    
    # Diccionarios de toda la muestra    
    erps = {'exparamilitar_co':exparamilitar_co_list, 'exparamilitar_in':exparamilitar_in_list, 'exguerrilla_co':exguerrilla_co_list, 'exguerrilla_in':exguerrilla_in_list, 'victim_co':victim_co_list, 'victim_in':victim_in_list}
    erps_type = {'victim_co':victim_co_list, 'victim_in':victim_in_list, 'excombatant_co':excombatant_co_list, 'excombatant_in':excombatant_in_list}
    erps_dif = {'exparamilitar_dif':exparamilitar_dif_list, 'exguerrilla_dif':exguerrilla_dif_list, 'victim_dif':victim_dif_list}
    erps_dif_type = {'victim_dif':victim_dif_list, 'excombatant_dif':excombatant_dif_list}
    return  erps, erps_dif, erps_type, erps_dif_type

In [24]:
# Configuro parámetros de gráficos
params = {'axes.labelsize':20, 'axes.titlesize': 20, 'axes.grid':True, 'axes.grid.axis':'both', 'axes.grid.which':'major','legend.fontsize':20, 'lines.linewidth': 2.0 ,'legend.loc': 'best','xtick.labelsize': 20, 'xtick.minor.visible': True, 'ytick.labelsize': 20, 'ytick.minor.visible': True}
plt.rcParams.update(params)

## Gráficas clústeres exparamilitares, exguerrilleros, víctimas

In [34]:
# Clúster 1
canales = ['POZ','PO3']
erps, __, __, __ = erps_grupos(canales=canales)
fig1 = mne.viz.plot_compare_evokeds(evokeds=erps, colors=['#1f77b4ff', '#ff7f0eff', '#2ca02cff', '#d62728ff', '#9467bdff', '#8c564bff'], 
                                    linestyles=['solid', 'solid','dashdot', 'dashdot', 'dashed', 'dashed'], axes=None, ci=None, truncate_yaxis=False, 
                                    truncate_xaxis=False, show_sensors=False, legend='upper left', split_legend=False, combine=None, show=False)
ax = fig1[0].gca()
rect1 = plt.Rectangle((0.09,-1.8), 0.05, 3.6, color='lightcyan')
ax.add_patch(rect1)

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [36]:
# Clúster 2
canales = ['CZ', 'FCZ', 'FC1', 'C1']
erps, __, __, __ = erps_grupos(canales=canales)
fig2 = mne.viz.plot_compare_evokeds(evokeds=erps, colors=['#1f77b4ff', '#ff7f0eff', '#2ca02cff', '#d62728ff', '#9467bdff', '#8c564bff'], 
                                    linestyles=['solid', 'solid','dashdot', 'dashdot', 'dashed', 'dashed'], axes=None, ci=None, truncate_yaxis=False, 
                                    truncate_xaxis=False, show_sensors=False, legend='lower left', split_legend=False, combine=None, show=False)
ax2 = fig2[0].gca()
rect2 = plt.Rectangle((0.31,-1.8), 0.05, 3.6, color='lightcyan')
ax2.add_patch(rect2)

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [37]:
# Clúster 3
canales = ['F1', 'FZ', 'F3']
erps, __, __, __ = erps_grupos(canales=canales)
fig3 = mne.viz.plot_compare_evokeds(evokeds=erps, colors=['#1f77b4ff', '#ff7f0eff', '#2ca02cff', '#d62728ff', '#9467bdff', '#8c564bff'],
                                    linestyles=['solid', 'solid','dashdot', 'dashdot', 'dashed', 'dashed'], axes=None, ci=None, truncate_yaxis=False,
                                    truncate_xaxis=False, show_sensors=False, legend='lower left', split_legend=False, combine=None, show=False)
ax3 = fig3[0].gca()
rect3 = plt.Rectangle((0.34,-1.8), 0.13, 3.6, color='lightcyan')
ax3.add_patch(rect3)

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [38]:
# Clúster 4
canales = ['FC6', 'C6', 'FC4']
erps, __, __, __ = erps_grupos(canales=canales)
fig4 = mne.viz.plot_compare_evokeds(evokeds=erps, colors=['#1f77b4ff', '#ff7f0eff', '#2ca02cff', '#d62728ff', '#9467bdff', '#8c564bff'],
                                    linestyles=['solid', 'solid','dashdot', 'dashdot', 'dashed', 'dashed'], axes=None, ci=None, truncate_yaxis=False,
                                    truncate_xaxis=False, show_sensors=False, legend='lower left', split_legend=False, combine=None, show=False)
ax4 = fig4[0].gca()
rect4 = plt.Rectangle((0.68,-1.8), 0.12, 3.6, color='lightcyan')
ax4.add_patch(rect4)

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [40]:
# Clúster 5
canales = ['P1', 'P3']
erps, __, __, __ = erps_grupos(canales=canales)
fig5 = mne.viz.plot_compare_evokeds(evokeds=erps, colors=['#1f77b4ff', '#ff7f0eff', '#2ca02cff', '#d62728ff', '#9467bdff', '#8c564bff'],
                                    linestyles=['solid', 'solid','dashdot', 'dashdot', 'dashed', 'dashed'], axes=None, ci=None, truncate_yaxis=False,
                                    truncate_xaxis=False, show_sensors=False, legend='upper left', split_legend=False, combine=None, show=False)
ax5 = fig5[0].gca()
rect5 = plt.Rectangle((0.72,-1.8), 0.08, 3.6, color='lightcyan')
ax5.add_patch(rect5)

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


In [44]:
# Clúster 6
canales = ['F6', 'F8']
erps, __, __, __ = erps_grupos(canales=canales)
fig6 = mne.viz.plot_compare_evokeds(evokeds=erps, colors=['#1f77b4ff', '#ff7f0eff', '#2ca02cff', '#d62728ff', '#9467bdff', '#8c564bff'],
                                    linestyles=['solid', 'solid','dashdot', 'dashdot', 'dashed', 'dashed'], axes=None, ci=None, truncate_yaxis=False,
                                    truncate_xaxis=False, show_sensors=False, legend='lower left', split_legend=False, combine=None, show=False)
ax6 = fig6[0].gca()
rect6 = plt.Rectangle((0.75,-1.8), 0.05, 3.6, color='lightcyan')
ax6.add_patch(rect6)

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


## Gráficas excombatientes, víctimas

In [45]:
# Clúster 1
canales = ['F2', 'FZ', 'F1', 'AF3', 'F3']
__, __, erps, __ = erps_grupos(canales=canales)
fig1 = mne.viz.plot_compare_evokeds(evokeds=erps, colors=['#1f77b4ff', '#ff7f0eff', '#2ca02cff', '#d62728ff', '#9467bdff', '#8c564bff'], 
                                    linestyles=['solid', 'solid','dashdot', 'dashdot', 'dashed', 'dashed'], axes=None, ci=None, truncate_yaxis=False, 
                                    truncate_xaxis=False, show_sensors=False, legend='upper left', split_legend=False, combine=None, show=False)
ax = fig1[0].gca()
rect1 = plt.Rectangle((0.34,-1.8), 0.13, 3.6, color='lightcyan')
ax.add_patch(rect1)

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.


# Gráficas por niveles de efecto IAT

In [8]:
# Matrices de muestra de sujetos a analizar
muestra = pd.read_csv('muestra_aleatoria_sin_ambos.csv')
negative = list(muestra[muestra['prejudice_level']=='negative']['subject'])
neutral = list(muestra[muestra['prejudice_level']=='neutral']['subject'])
positive = list(muestra[muestra['prejudice_level']=='positive']['subject'])
print(negative)

[23009, 22111, 23013, 21122, 22100, 22116, 22110, 21113, 22113, 21140, 23002, 21139, 23016, 23017, 23006, 21118, 22107, 21123, 22108]


In [2]:
# Gráfica de densidad
import seaborn as sns
# Configuro parámetros de gráficos
params = {'axes.labelsize':20, 'axes.titlesize': 20, 'axes.grid':True, 'axes.grid.axis':'both', 'axes.grid.which':'both','legend.fontsize':20, 'legend.loc': 'best', 'legend.title_fontsize':20, 'xtick.labelsize': 20, 'xtick.minor.visible': True, 'ytick.labelsize': 20, 'ytick.minor.visible': True}
plt.rcParams.update(params)
muestra = pd.read_csv('muestra_aleatoria_sin_ambos.csv')
muestra.rename(columns={'dscore_4':'IAT score', 'prejudice_level':'IAT effect level group'}, inplace=True)
density_plot = sns.kdeplot(data=muestra, x='IAT score', hue='IAT effect level group', fill=True, alpha=0.3)
density_plot.set_alpha=0.3

In [9]:
for i in range(len(negative)):
    negative[i] = str(negative[i])
for i in range(len(neutral)):
    neutral[i] = str(neutral[i])
for i in range(len(positive)):
    positive[i] = str(positive[i])
__, __, negative_co, negative_in, __, __, __ = matrices_cluster(negative)
__, __, neutral_co, neutral_in, __, __, __ = matrices_cluster(neutral)
__, __, positive_co, positive_in, tiempos, canales, adjacency = matrices_cluster(positive)

# Matrices de diferencias entre ensayos
negative_dif = negative_co - negative_in
neutral_dif = neutral_co - neutral_in
positive_dif = positive_co - positive_in
print(np.shape(positive_dif))

Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(19, 256, 64)
(19, 256, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(37, 256, 64)
(37, 256, 64)
Could not find a adjacency matrix for the data. Computing adjacency based on Delaunay triangulations.
-- number of adjacent vertices : 64
Identifying common channels ...
Identifying common channels ...
(29, 256, 64)
(29, 256, 64)
(29, 256, 64)


In [10]:
def lista_evocados_niveles(canales, tmin=0.0, tmax=0.796875):
    negative_co = {}
    negative_in = {}
    negative_dif = {}
    for sujeto in negative:
        negative_co[sujeto] = ROI_evoked(evocados_co[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        negative_in[sujeto] = ROI_evoked(evocados_in[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        data_dif = negative_co[sujeto].data - negative_in[sujeto].data
        negative_dif[sujeto] = mne.EvokedArray(data=data_dif, info=negative_in[sujeto].info, tmin=negative_in[sujeto].tmin)
    neutral_co = {}
    neutral_in = {}
    neutral_dif = {}
    for sujeto in neutral:
        neutral_co[sujeto] = ROI_evoked(evocados_co[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        neutral_in[sujeto] = ROI_evoked(evocados_in[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        data_dif = neutral_co[sujeto].data - neutral_in[sujeto].data
        neutral_dif[sujeto] = mne.EvokedArray(data=data_dif, info=neutral_in[sujeto].info, tmin=neutral_in[sujeto].tmin)
    positive_co = {}
    positive_in = {}
    positive_dif = {}
    for sujeto in positive:
        positive_co[sujeto] = ROI_evoked(evocados_co[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        positive_in[sujeto] = ROI_evoked(evocados_in[sujeto],canales).crop(tmin=tmin,tmax=tmax)
        data_dif = positive_co[sujeto].data - positive_in[sujeto].data
        positive_dif[sujeto] = mne.EvokedArray(data=data_dif, info=positive_in[sujeto].info, tmin=positive_in[sujeto].tmin)

    return negative_co, neutral_co, positive_co, negative_in, neutral_in, positive_in,negative_dif,neutral_dif,positive_dif

In [14]:
def erps_niveles(canales, tmin=0.0, tmax=0.796875):
    # Diccionarios de evocados de clústeres por grupos
    negative_co,neutral_co,positive_co,negative_in,neutral_in,positive_in,negative_dif,neutral_dif,positive_dif = lista_evocados_niveles(canales=canales, tmin=tmin, tmax=tmax)

    # Diccionarios de toda la muestra
    ROI_co = {**negative_co,**neutral_co,**positive_co}
    ROI_in = {**negative_in,**neutral_in,**positive_in}
    ROI_dif = {**negative_dif,**neutral_dif,**positive_dif}

    # Listas de evocados de clústeres por grupos
    negative_co_list = list(negative_co.values())
    neutral_co_list = list(neutral_co.values())
    positive_co_list = list(positive_co.values())
    negative_in_list = list(negative_in.values())
    neutral_in_list = list(neutral_in.values())
    positive_in_list = list(positive_in.values())
    negative_dif_list = list(negative_dif.values())
    neutral_dif_list = list(neutral_dif.values())
    positive_dif_list = list(positive_dif.values())

    erps = {'negative_co':negative_co_list, 'neutral_co':neutral_co_list, 'positive_co':positive_co_list, 'negative_in':negative_in_list, 'neutral_in':neutral_in_list, 'positive_in':positive_in_list}
    erps_dif = {'negative_dif':negative_dif_list, 'neutral_dif':neutral_dif_list, 'positive_dif':positive_dif_list}
    return  erps, erps_dif

In [18]:
# Configuro parámetros de gráficos
params = {'axes.labelsize':20, 'axes.titlesize': 20, 'axes.grid':True, 'axes.grid.axis':'both', 'axes.grid.which':'both','legend.fontsize':20, 'lines.linewidth': 2.0 ,'legend.loc': 'lower left','xtick.labelsize': 20, 'xtick.minor.visible': True, 'ytick.labelsize': 20, 'ytick.minor.visible': True}
plt.rcParams.update(params)

In [15]:
canales = ['FZ', 'FC1', 'FC2']
erps, erps_dif = erps_niveles(canales=canales)
fig1 = mne.viz.plot_compare_evokeds(evokeds=erps, colors=['#1f77b4ff', '#ff7f0eff', '#2ca02cff', '#d62728ff', '#9467bdff', '#8c564bff'], linestyles=['solid', 'dashdot', 'dashed', 'solid', 'dashdot', 'dashed'], axes=None, ci=None, truncate_yaxis=False, truncate_xaxis=False, show_sensors=False, legend='lower left', split_legend=False, combine=None, show=False)
ax = fig1[0].gca()
rect1 = plt.Rectangle((0.14,-1.8), 0.09, 3.6, color='lightcyan')
ax.add_patch(rect1)
rect2 = plt.Rectangle((0.29,-1.8), 0.08, 3.6, color='lightcyan')
ax.add_patch(rect2)
rect3 = plt.Rectangle((0.46,-1.8), 0.06, 3.6, color='lightcyan')
ax.add_patch(rect3)
rect4 = plt.Rectangle((0.54,-1.8), 0.26, 3.6, color='lightcyan')
ax.add_patch(rect4)

Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode: mean)
Applying baseline correction (mode

In [20]:
fig2 = mne.viz.plot_compare_evokeds(evokeds=erps_dif, colors=['#1f77b4ff', '#ff7f0eff', '#2ca02cff'], linestyles=['solid', 'dashdot', 'dashed'],axes=None, ci=None, truncate_yaxis=False, truncate_xaxis=False, show_sensors=False, legend='lower left', split_legend=False, combine=None, show=False)
ax = fig2[0].gca()
rect1 = plt.Rectangle((0.14,-1.8), 0.09, 3.6, color='lightcyan')
ax.add_patch(rect1)
rect2 = plt.Rectangle((0.29,-1.8), 0.08, 3.6, color='lightcyan')
ax.add_patch(rect2)
rect3 = plt.Rectangle((0.46,-1.8), 0.06, 3.6, color='lightcyan')
ax.add_patch(rect3)
rect4 = plt.Rectangle((0.54,-1.8), 0.26, 3.6, color='lightcyan')
ax.add_patch(rect4)

The PostScript backend does not support transparency; partially transparent artists will be rendered opaque.
